In [1]:
import sys
#for creating the mapper code
from sqlalchemy import Column, ForeignKey, Integer, String, Float

#for configuration and class code
from sqlalchemy.ext.declarative import declarative_base

#for creating foreign key relationship between the tables
from sqlalchemy.orm import relationship

#for configuration
from sqlalchemy import create_engine

#create declarative_base instance
Base = declarative_base()

#we'll add classes here

#creates a create_engine instance at the bottom of the file
engine = create_engine('sqlite:///transcriptomics.db')

# Base.metadata.create_all(engine)

In [2]:
from sqlalchemy.orm import load_only

In [2]:
import GEOparse
import re
import os
import pandas as pd
import numpy as np
from scipy import stats
from GSEpipeline import *

# find project root dir
currentdir = os.getcwd()
dirlist = currentdir.split('/')
projectdir = '/'.join(dirlist[0:-1])

In [4]:
class Sample(Base):
    __tablename__ = 'sample'

    id = Column(Integer, primary_key=True)
    ENTREZ_GENE_ID = Column(String(250), nullable=False)
    VALUE = Column(Float, nullable=False)
    P_VALUE = Column(Float)
    ABS_CALL = Column(String(2))
    Sample = Column(String(64))
    
    
class IDMapps(Base):
    __tablename__ = 'id_entrez_map'
    
    idx = Column(Integer, primary_key=True)
    ID = Column(String(250))
    ENTREZ = Column(String(250))

class GSEinfo(Base):
    __tablename__ = 'gseinfo'
    
    Sample = Column(String(64), primary_key=True)
    GSE = Column(String(64))
    Platform = Column(String(64))

In [5]:
Base.metadata.create_all(engine)

In [5]:
# create instance of project
gse2770 = GSEproject('GSE2770',projectdir)

14-Aug-2019 13:34:45 INFO GEOparse - Parsing ./GSE2770_family.soft.gz: 
14-Aug-2019 13:34:45 DEBUG GEOparse - DATABASE: GeoMiame
14-Aug-2019 13:34:45 DEBUG GEOparse - SERIES: GSE2770
14-Aug-2019 13:34:45 DEBUG GEOparse - PLATFORM: GPL96


Initialize project (GSE2770):
Root: /Users/zhzhao/Dropbox/Helikar/pipelines
Raw data: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW


14-Aug-2019 13:34:47 DEBUG GEOparse - PLATFORM: GPL97
14-Aug-2019 13:34:48 DEBUG GEOparse - PLATFORM: GPL8300
14-Aug-2019 13:34:48 DEBUG GEOparse - SAMPLE: GSM60348
14-Aug-2019 13:34:48 DEBUG GEOparse - SAMPLE: GSM60349
14-Aug-2019 13:34:48 DEBUG GEOparse - SAMPLE: GSM60350
14-Aug-2019 13:34:48 DEBUG GEOparse - SAMPLE: GSM60351
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60352
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60353
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60354
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60355
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60356
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60357
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60358
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60359
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60360
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60361
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60362
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60363
14-Aug-201

GPL96
[HG-U133A] Affymetrix Human Genome U133A Array
GPL97
[HG-U133B] Affymetrix Human Genome U133B Array
GPL8300
[HG_U95Av2] Affymetrix Human Genome U95 Version 2 Array


In [6]:
df_clean = gse2770.get_entrez_table_pipeline(fromcsv=True)
df_clean.sort_index(inplace=True)
df_clean

/usr/local/Cellar/ipython/7.5.0/libexec/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (101,104,107,110,113,116,119,122,125,128,131,134,137,140,143,146,149,152,155,158,161,164,167,170,173,176,179,182,185,188,191,194,197,200) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


,ENTREZ_GENE_ID,gsm60349.cel.gz,gsm60349.cel.gz.1,gsm60349.cel.gz.2,gsm60350.cel.gz,gsm60350.cel.gz.1,gsm60350.cel.gz.2,gsm60351.cel.gz,gsm60351.cel.gz.1,gsm60351.cel.gz.2,...,gsm60763.cel.gz.2,gsm60764.cel.gz,gsm60764.cel.gz.1,gsm60764.cel.gz.2,gsm60765.cel.gz,gsm60765.cel.gz.1,gsm60765.cel.gz.2,gsm60766.cel.gz,gsm60766.cel.gz.1,gsm60766.cel.gz.2
0,780 /// 100616237,717.599098,P,0.017000,966.038612,P,0.003067,724.167413,P,0.005643,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5982,140.340345,A,0.204022,124.764102,A,0.107301,90.986765,A,0.250724,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3310,68.989968,A,0.458816,55.205940,A,0.302547,97.283915,A,0.175989,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7849,1281.802247,P,0.017000,902.213482,P,0.019303,919.399037,P,0.027860,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2978,200.536466,A,0.218983,277.363337,A,0.060419,300.050853,A,0.117161,...,0.143002,373.110629,A,0.164032,626.187096,A,0.186972,401.599221,A,0.186972
5,7318 /// 100847079,540.953164,P,0.021866,255.370653,P,0.031336,506.635753,P,0.013092,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7067,223.734251,P,0.027860,158.550443,P,0.021866,189.557546,A,0.189687,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,11099,65.106109,A,0.541184,64.526428,A,0.602006,49.510810,A,0.602006,...,0.570568,90.043163,A,0.535424,114.817064,A,0.605162,89.731430,A,0.638946
8,6352,217.342576,P,0.009985,285.981770,P,0.000388,214.198185,P,0.035163,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1571,126.325873,P,0.039365,53.677210,A,0.418069,55.656157,A,0.267463,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Write to DATABASE

In [7]:
df_samples = pd.DataFrame([],columns=['ENTREZ_GENE_ID','VALUE','P_VALUE','ABS_CALL','Sample'])
df_samples.index.name = 'id'

for key,val in gse2770.gsm_platform.items():
    col_val = '{}.cel.gz'.format(key.lower())
    col_abs = '{}.cel.gz.1'.format(key.lower())
    col_p = '{}.cel.gz.2'.format(key.lower())
    if not col_val in list(df_clean):
        continue
    df_s = pd.DataFrame([])
    df_s.index.name='id'
    df_s['ENTREZ_GENE_ID'] = df_clean['ENTREZ_GENE_ID']
    df_s['VALUE'] = df_clean[col_val]
    df_s['ABS_CALL'] = df_clean[col_abs]
    df_s['P_VALUE'] = df_clean[col_p]
    df_s['Sample'] = key.upper()
    df_samples = pd.concat([df_samples, df_s.dropna(how='any')], ignore_index=True)

df_gseinfo = pd.DataFrame([],columns=['Sample','GSE','Platform'])
df_gseinfo['Sample'] = pd.Series(list(gse2770.gsm_platform.keys()))
df_gseinfo['Platform'] = pd.Series(list(gse2770.gsm_platform.values()))
df_gseinfo['GSE'] = gse2770.gsename
# df_database.dropna(how='any',inplace=True)
# df_database.reset_index(inplace=True)

/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [10]:
df_samples.to_sql(con=engine,name='sample',if_exists='replace',index_label='id')

In [14]:
df_gseinfo.to_sql(con=engine,name='gseinfo',if_exists='replace',index=False)

In [29]:
df = pd.read_sql(session.query(GSEinfo).filter().statement, session.bind)
df

,Sample,GSE,Platform
0,GSM565273,GSE22886,GPL96
1,GSM565274,GSE22886,GPL96
2,GSM565275,GSE22886,GPL96
3,GSM565290,GSE22886,GPL96
4,GSM565291,GSE22886,GPL96
5,GSM565292,GSE22886,GPL96
6,GSM548000,GSE22045,GPL570
7,GSM548001,GSE22045,GPL570
8,GSM607510,GSE24634,GPL96
9,GSM607511,GSE24634,GPL96


In [30]:
df = pd.read_sql(session.query(Sample).filter().statement, session.bind, index_col='id')
df

,ENTREZ_GENE_ID,VALUE,P_VALUE,ABS_CALL,Sample
id,,,,,
1,780 /// 100616237,419.118881,0.020430,P,GSM565273
2,5982,319.910441,0.266847,A,GSM565273
3,3310,580.386943,0.008184,P,GSM565273
4,7849,1628.834605,0.013092,P,GSM565273
5,2978,130.252482,0.123873,A,GSM565273
6,7318 /// 100847079,1487.899054,0.025230,P,GSM565273
7,7067,240.979262,0.089405,A,GSM565273
8,11099,53.679802,0.266847,A,GSM565273
9,6352,10180.161103,0.001673,P,GSM565273


In [6]:
from sqlalchemy.orm import sessionmaker

DBSession = sessionmaker(bind=engine)

session = DBSession()

In [32]:
aaa = session.query(Book).all()
aaa

 ...]

In [17]:
inqueries[1].fillna(method='ffill',inplace=True)
inqueries[1]

,GSE ID,Samples,Source,Experiment,Time,GPL ID,Instrument
0,GSE2770,GSM60353,cord blood CD4+ cells,antiCD3+andtiCD28+IL4,2 hours,GPL8300,affy
1,GSE2770,GSM60354,cord blood CD4+ cells,antiCD3+andtiCD28+IL4,6 hours,GPL8300,affy
2,GSE2770,GSM60360,cord blood CD4+ cells,antiCD3+andtiCD28+IL4,48 hours,GPL8300,affy
3,GSE2770,GSM60367,cord blood CD4+ cells,antiCD3+andtiCD28+IL4,48 hours,GPL8300,affy
4,GSE2770,GSM60377,cord blood CD4+ cells,antiCD3+andtiCD28+IL4,6 hours,GPL8300,affy
5,GSE2770,GSM60381,cord blood CD4+ cells,antiCD3+andtiCD28+IL4,2 hours,GPL8300,affy
6,GSE2770,GSM60738,cord blood CD4+ cells,antiCD3+andtiCD28+IL4,2 hours,GPL97,affy
7,GSE2770,GSM60739,cord blood CD4+ cells,antiCD3+andtiCD28+IL4,6 hours,GPL97,affy
8,GSE2770,GSM60745,cord blood CD4+ cells,antiCD3+andtiCD28+IL4,48 hours,GPL97,affy
9,GSE2770,GSM60752,cord blood CD4+ cells,antiCD3+andtiCD28+IL4,48 hours,GPL97,affy


In [20]:
# function to complete the inquery of a sheet
def queryTest(df):
    sr = df['GSE ID']
    gse_ids = sr[sr.str.match('GSE')].unique()
    sr = df['Samples'].dropna()
    gsm_ids = sr.unique()
    print('Start Collecting Data for:')
    print(gse_ids)
    print(gsm_ids)
    

In [23]:
# input from user
filename = 'GeneExpressionDataUsed.xlsx'
sheet_name = list(range(5)) # first 5 sheets

inqueryFullPath = os.path.join(projectdir,'data', filename)
inqueries = pd.read_excel(inqueryFullPath, sheet_name=sheet_name, header=0)

for i in range(5):
    # print(list(inqueries[i]))
    inqueries[i].fillna(method='ffill',inplace=True)
    df = inqueries[i].loc[:,['GSE ID','Samples','GPL ID','Instrument']]
    queryTest(df)

Start Collecting Data for:
['GSE22886' 'GSE43005' 'GSE22045' 'GSE24634']
['GSM565273' 'GSM565274' 'GSM565275' 'GSM565290' 'GSM565291' 'GSM565292'
 'GSM1054773' 'GSM1054779' 'GSM1054781' 'GSM1054789' 'GSM548000'
 'GSM548001' 'GSM607510' 'GSM607511' 'GSM607512']
Start Collecting Data for:
['GSE2770' 'GSE22886' 'GSE43769' 'GSE43005' 'GSE50175' 'GSE30664']
['GSM60353' 'GSM60354' 'GSM60360' 'GSM60367' 'GSM60377' 'GSM60381'
 'GSM60738' 'GSM60739' 'GSM60745' 'GSM60752' 'GSM60762' 'GSM60766'
 'GSM60704' 'GSM60705' 'GSM60711' 'GSM60718' 'GSM60728' 'GSM60732'
 'GSM565993' 'GSM565994' 'GSM565995' 'GSM565996' 'GSM565997' 'GSM565276'
 'GSM565277' 'GSM565278' 'GSM565279' 'GSM565280' 'GSM1070758' 'GSM1070754'
 'GSM1070762' 'GSM1054775' 'GSM1054783' 'GSM1054787' 'GSM1054791'
 'GSM1215107' 'GSM1215109' 'GSM1215111' 'GSM1215113' 'GSM760599'
 'GSM760601']
Start Collecting Data for:
['GSE2770' 'GSE22886' 'GSE43769' 'GSE43005']
['GSM60357' 'GSM60358' 'GSM60362' 'GSM60368' 'GSM60373' 'GSM60379'
 'GSM60708' 

In [178]:
df_results = pd.DataFrame([],columns=['ENTREZ_GENE_ID'])
df_results.set_index('ENTREZ_GENE_ID',inplace=True)
for gsm in gsm_ids:
    #print(gsm)
    df = pd.read_sql(session.query(Sample).filter_by(Sample=gsm).options(load_only("ABS_CALL","ENTREZ_GENE_ID")).statement, 
                     session.bind, 
                     index_col='ENTREZ_GENE_ID')
    df.drop('id',axis=1,inplace=True)
    df.rename(columns={'ABS_CALL':gsm},inplace=True)
    df.loc[df[gsm]=='A',gsm] = 0
    df.loc[df[gsm]=='P',gsm] = 1
    df.loc[df[gsm]=='M',gsm] = 1
    
    df_results = pd.concat([df_results, df], axis=1, sort=False)

# Need to set index name after merge
df_results.index.name = 'ENTREZ_GENE_ID'
df_results

,GSM60353,GSM60354,GSM60360,GSM60367,GSM60377,GSM60381,GSM60738,GSM60739,GSM60745,GSM60752,...,GSM1054775,GSM1054783,GSM1054787,GSM1054791,GSM1215107,GSM1215109,GSM1215111,GSM1215113,GSM760599,GSM760601
ENTREZ_GENE_ID,,,,,,,,,,,,,,,,,,,,,
780 /// 100616237,1.0,1.0,0.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5982,0.0,0.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3310,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7849,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7318 /// 100847079,1.0,1.0,1.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7067,1.0,0.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11099,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6352,0.0,1.0,1.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
# df_results[['///' in s for s in df_results.index]]
df_results[df_results.index.str.contains('///')]
# df_results

,GSM60353,GSM60354,GSM60360,GSM60367,GSM60377,GSM60381,GSM60738,GSM60739,GSM60745,GSM60752,...,GSM1054775,GSM1054783,GSM1054787,GSM1054791,GSM1215107,GSM1215109,GSM1215111,GSM1215113,GSM760599,GSM760601
ENTREZ_GENE_ID,,,,,,,,,,,,,,,,,,,,,
780 /// 100616237,1.0,1.0,0.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7318 /// 100847079,1.0,1.0,1.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2342 /// 100529261,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6158 /// 102465483,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6144 /// 26771 /// 619499 /// 100131205,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3178 /// 664709,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6207 /// 85388 /// 100508408,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6205 /// 84546,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1655 /// 100616387 /// 100616408,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
# df = pd.read_sql(session.query(GSEinfo).filter_by(Sample='GSM60728').statement, session.bind)
# print(df)
# df = pd.read_sql(session.query(Sample).filter_by(Sample='GSM60728').statement, 
#                  session.bind, 
#                  index_col='ENTREZ_GENE_ID',
#                  columns=['ABS_CALL']
#                 )
df = pd.read_sql(session.query(Sample).filter_by(Sample='GSM60728').options(load_only("ABS_CALL","ENTREZ_GENE_ID")).statement, 
                 session.bind, 
                 index_col='ENTREZ_GENE_ID'
                )
df.drop('id',axis=1,inplace=True)
print(df)

                                                 ABS_CALL
ENTREZ_GENE_ID                                           
780 /// 100616237                                       P
5982                                                    A
3310                                                    A
7849                                                    P
2978                                                    A
7318 /// 100847079                                      A
7067                                                    P
11099                                                   A
6352                                                    P
1571                                                    A
2049                                                    A
2101                                                    P
1548                                                    A
2621                                                    A
4323                                                    P
8717          

In [75]:
import mygene
mg = mygene.MyGeneInfo()
ens = ['ENSG00000148795', 'ENSG00000165359', 'ENSG00000150676']
ginfo = mg.querymany(ens, scopes='ensembl.gene')

for g in ginfo:
    for k, v in g.items():
        print("- {0: <10}: {1}".format(k, v))
    print('')

querying 1-3...done.
Finished.
- query     : ENSG00000148795
- _id       : 1586
- _score    : 18.763334
- entrezgene: 1586
- name      : cytochrome P450 family 17 subfamily A member 1
- symbol    : CYP17A1
- taxid     : 9606

- query     : ENSG00000165359
- _id       : 203522
- _score    : 19.774754
- entrezgene: 203522
- name      : integrator complex subunit 6 like
- symbol    : INTS6L
- taxid     : 9606

- query     : ENSG00000150676
- _id       : 220047
- _score    : 21.06969
- entrezgene: 220047
- name      : coiled-coil domain containing 83
- symbol    : CCDC83
- taxid     : 9606



## Merge duplicated Entrez_GENE_ID

In [180]:
id_list = []
entrez_id_list = df_results[df_results.index.str.contains('///')].index.tolist()
for entrez_id in entrez_id_list:
    entrez_ids = entrez_id.split(' /// ')
    id_list.extend(entrez_ids)

print(len(id_list))
id_list

3871


['780',
 '100616237',
 '7318',
 '100847079',
 '2342',
 '100529261',
 '6158',
 '102465483',
 '6144',
 '26771',
 '619499',
 '100131205',
 '3178',
 '664709',
 '6207',
 '85388',
 '100508408',
 '6205',
 '84546',
 '1655',
 '100616387',
 '100616408',
 '6139',
 '26790',
 '26791',
 '497661',
 '100124516',
 '100526842',
 '7919',
 '692199',
 '100532737',
 '8683',
 '283459',
 '23020',
 '101929240',
 '375',
 '100500810',
 '3020',
 '3021',
 '440926',
 '6124',
 '595097',
 '595098',
 '595099',
 '595100',
 '6189',
 '8944',
 '7184',
 '100500842',
 '801',
 '805',
 '808',
 '9782',
 '724102',
 '6418',
 '646817',
 '8663',
 '728689',
 '10399',
 '619570',
 '619571',
 '1937',
 '100500804',
 '1933',
 '619569',
 '2280',
 '101929368',
 '23521',
 '26816',
 '26817',
 '26818',
 '26819',
 '728658',
 '6427',
 '693221',
 '5757',
 '728026',
 '100302285',
 '100422872',
 '100422885',
 '100506248',
 '3303',
 '3304',
 '3305',
 '6202',
 '26811',
 '94163',
 '6142',
 '26780',
 '10528',
 '26792',
 '692201',
 '692213',
 '1003021

In [181]:
entrez_id_list

['780 /// 100616237',
 '7318 /// 100847079',
 '2342 /// 100529261',
 '6158 /// 102465483',
 '6144 /// 26771 /// 619499 /// 100131205',
 '3178 /// 664709',
 '6207 /// 85388 /// 100508408',
 '6205 /// 84546',
 '1655 /// 100616387 /// 100616408',
 '6139 /// 26790 /// 26791 /// 497661 /// 100124516 /// 100526842',
 '7919 /// 692199 /// 100532737',
 '8683 /// 283459',
 '23020 /// 101929240',
 '375 /// 100500810',
 '3020 /// 3021 /// 440926',
 '6124 /// 595097 /// 595098 /// 595099 /// 595100',
 '6189 /// 8944',
 '7184 /// 100500842',
 '801 /// 805 /// 808',
 '9782 /// 724102',
 '6418 /// 646817',
 '8663 /// 728689',
 '10399 /// 619570 /// 619571',
 '1937 /// 100500804',
 '1933 /// 619569',
 '2280 /// 101929368',
 '23521 /// 26816 /// 26817 /// 26818 /// 26819 /// 728658',
 '6427 /// 693221',
 '5757 /// 728026 /// 100302285 /// 100422872 /// 100422885 /// 100506248',
 '3303 /// 3304 /// 3305',
 '6202 /// 26811 /// 94163',
 '6142 /// 26780',
 '10528 /// 26792 /// 692201 /// 692213 /// 1003021

In [182]:
entrez_single_id_list = df_results[~df_results.index.str.contains('///')].index.tolist()
entrez_single_id_list

['5982',
 '3310',
 '7849',
 '2978',
 '7067',
 '11099',
 '6352',
 '1571',
 '2049',
 '2101',
 '1548',
 '2621',
 '4323',
 '8717',
 '5337',
 '441263',
 '572',
 '10594',
 '826',
 '11224',
 '1982',
 '8664',
 '11315',
 '6727',
 '2665',
 '6135',
 '377',
 '6152',
 '3183',
 '4735',
 '6233',
 '2197',
 '23435',
 '1072',
 '6141',
 '8665',
 '6193',
 '6155',
 '6164',
 '4677',
 '56910',
 '6143',
 '5250',
 '6133',
 '6128',
 '23399',
 '4736',
 '11335',
 '5690',
 '10657',
 '51493',
 '2079',
 '23',
 '1603',
 '7528',
 '10899',
 '11143',
 '7705',
 '9092',
 '3608',
 '9552',
 '8882',
 '6881',
 '10438',
 '4841',
 '387',
 '10921',
 '6229',
 '6156',
 '4869',
 '3326',
 '3550',
 '8724',
 '821',
 '9733',
 '27013',
 '10285',
 '4670',
 '3184',
 '9045',
 '2987',
 '79036',
 '4946',
 '533',
 '3735',
 '6194',
 '6201',
 '23326',
 '10944',
 '6923',
 '1327',
 '10959',
 '2339',
 '6230',
 '6167',
 '3094',
 '1938',
 '6204',
 '8992',
 '3190',
 '51433',
 '3192',
 '8661',
 '4478',
 '81',
 '351',
 '5573',
 '1832',
 '5885',
 '9948'

In [184]:
common_elements = list(set(entrez_single_id_list).intersection(set(id_list)))
print(len(common_elements))

905


In [201]:
print(len(common_elements))
print('id_list: {}, set: {}'.format(len(id_list),len(set(id_list))))
print('entrez_single_id_list: {}, set: {}'.format(len(entrez_single_id_list),len(set(entrez_single_id_list))))
print('entrez_id_list: {}, set: {}'.format(len(entrez_id_list),len(set(entrez_id_list))))
dups = [x for x in id_list if id_list.count(x) > 1]
print('dups: {}, set: {}'.format(len(dups),len(set(dups))))
dups = set(dups).union(common_elements)

905
id_list: 3871, set: 2884
entrez_single_id_list: 18229, set: 18229
entrez_id_list: 1356, set: 1356
dups: 1527, set: 540


In [214]:
full_entre_id_sets = []
cnt = 0
entrez_dups_list = []
for dup_id in dups:
    print(dup_id+':')
    id_set = []
    entrez_dups = []
    for multi_ids in entrez_id_list:
        if dup_id in multi_ids.split(' /// '):
            print('{}'.format(multi_ids))
            id_set.extend(multi_ids.split(' /// '))
            entrez_dups.append(multi_ids)
    id_set = list(set(id_set))
    id_set.sort(key=int)
    entrez_dups.extend(id_set)
    full_entre_id = ' /// '.join(id_set)
    print('Merged {}: {}\n'.format(dup_id,full_entre_id))
    full_entre_id_sets.append(full_entre_id)
    entrez_dups_list.append(entrez_dups)
    cnt+=1

entrez_dups_dict = dict(zip(full_entre_id_sets,entrez_dups_list))
full_entre_id_sets = list(set(full_entre_id_sets))

284757:
284757 /// 729296
Merged 284757: 284757 /// 729296

1485:
1485 /// 246100
1485 /// 30848 /// 246100
Merged 1485: 1485 /// 30848 /// 246100

5911:
5911 /// 5912
Merged 5911: 5911 /// 5912

80014:
53842 /// 80014
Merged 80014: 53842 /// 80014

54877:
54877 /// 400653
Merged 54877: 54877 /// 400653

100526832:
151230 /// 100526832
151230 /// 493911 /// 100526832
Merged 100526832: 151230 /// 493911 /// 100526832

100526693:
10093 /// 26140 /// 100526693
10093 /// 100526693
Merged 100526693: 10093 /// 26140 /// 100526693

8906:
8906 /// 84502
Merged 8906: 8906 /// 84502

4763:
4763 /// 101930150
Merged 4763: 4763 /// 101930150

100131998:
100131998 /// 101060405
Merged 100131998: 100131998 /// 101060405

101929792:
9659 /// 100996724 /// 100996761 /// 101929792 /// 101930416
9659 /// 653513 /// 100996724 /// 100996761 /// 101929792 /// 101930416
Merged 101929792: 9659 /// 653513 /// 100996724 /// 100996761 /// 101929792 /// 101930416

101930655:
647087 /// 101930655
26266 /// 647087

23049 /// 81893 /// 440345 /// 440354 /// 552900 /// 595101 /// 613037 /// 641298 /// 100132247 /// 100271836 /// 100288332 /// 100506060 /// 101060386 /// 101060596 /// 101060604 /// 102723773
23117 /// 440345 /// 613037 /// 728888 /// 100132247 /// 100288332 /// 101060275 /// 101929910
23117 /// 440345 /// 613037 /// 728734 /// 728888 /// 100132247 /// 100288332 /// 101929910
Merged 613037: 9284 /// 23049 /// 23117 /// 81893 /// 283970 /// 339044 /// 399491 /// 440345 /// 440354 /// 552900 /// 595101 /// 613037 /// 641298 /// 642778 /// 642799 /// 728734 /// 728741 /// 728888 /// 100132247 /// 100271836 /// 100288332 /// 100506060 /// 100507607 /// 101059938 /// 101059953 /// 101060275 /// 101060386 /// 101060596 /// 101060604 /// 101929910 /// 102723773 /// 102725125

486:
486 /// 100533181
Merged 486: 486 /// 100533181

81853:
51522 /// 81853
Merged 81853: 51522 /// 81853

26012:
26012 /// 102466223
Merged 26012: 26012 /// 102466223

145781:
81488 /// 145781
81488 /// 145781 /// 10


4151:
4151 /// 284912
Merged 4151: 4151 /// 284912

259307:
23636 /// 259307
Merged 259307: 23636 /// 259307

56145:
56145 /// 56146
9752 /// 56134 /// 56135 /// 56136 /// 56137 /// 56138 /// 56139 /// 56140 /// 56141 /// 56142 /// 56143 /// 56144 /// 56145 /// 56146 /// 56147
Merged 56145: 9752 /// 56134 /// 56135 /// 56136 /// 56137 /// 56138 /// 56139 /// 56140 /// 56141 /// 56142 /// 56143 /// 56144 /// 56145 /// 56146 /// 56147

653238:
2966 /// 653238 /// 728340 /// 730394
2966 /// 653238 /// 728340
2966 /// 653238
Merged 653238: 2966 /// 653238 /// 728340 /// 730394

3316:
3316 /// 91894 /// 100528019
Merged 3316: 3316 /// 91894 /// 100528019

729447:
2543 /// 2574 /// 2576 /// 2577 /// 2578 /// 2579 /// 26748 /// 26749 /// 645051 /// 645073 /// 729396 /// 729408 /// 729422 /// 729428 /// 729431 /// 729442 /// 729447 /// 100008586 /// 100101629 /// 100132399
2543 /// 2574 /// 26749 /// 645037 /// 645051 /// 729408 /// 729447 /// 100101629
2543 /// 2574 /// 2576 /// 2577 /// 257

6477 /// 83752 /// 100507577
Merged 6477: 6477 /// 83752 /// 100507577

56981:
56981 /// 101060179
Merged 56981: 56981 /// 101060179

388799:
200232 /// 388799
Merged 388799: 200232 /// 388799

8368:
8294 /// 8359 /// 8360 /// 8361 /// 8362 /// 8363 /// 8364 /// 8365 /// 8366 /// 8367 /// 8368 /// 8370 /// 121504 /// 554313
Merged 8368: 8294 /// 8359 /// 8360 /// 8361 /// 8362 /// 8363 /// 8364 /// 8365 /// 8366 /// 8367 /// 8368 /// 8370 /// 121504 /// 554313

2953:
2953 /// 653689
Merged 2953: 2953 /// 653689

56139:
9752 /// 56134 /// 56135 /// 56136 /// 56137 /// 56138 /// 56139 /// 56140 /// 56141 /// 56142 /// 56143 /// 56144 /// 56145 /// 56146 /// 56147
Merged 56139: 9752 /// 56134 /// 56135 /// 56136 /// 56137 /// 56138 /// 56139 /// 56140 /// 56141 /// 56142 /// 56143 /// 56144 /// 56145 /// 56146 /// 56147

10588:
10588 /// 100528021
10588 /// 400410 /// 100528021
Merged 10588: 10588 /// 400410 /// 100528021

2576:
2543 /// 2574 /// 2576 /// 2577 /// 2578 /// 2579 /// 26748 

3183 /// 343069 /// 440563 /// 649330 /// 101060301
Merged 3183: 3183 /// 343069 /// 440563 /// 649330 /// 101060301

730092:
54700 /// 653390 /// 730092 /// 101930306
54700 /// 653390 /// 730092
Merged 730092: 54700 /// 653390 /// 730092 /// 101930306

81488:
81488 /// 145781
81488 /// 145781 /// 100820829
Merged 81488: 81488 /// 145781 /// 100820829

102157402:
6880 /// 102157402
Merged 102157402: 6880 /// 102157402

101929857:
6799 /// 6817 /// 6818 /// 445329 /// 100526830 /// 100526831 /// 101929857
6818 /// 79008 /// 445329 /// 548593 /// 100526830 /// 100526831 /// 101929857 /// 101929864
6818 /// 445329 /// 100526830 /// 100526831 /// 101929857
6818 /// 79008 /// 445329 /// 548593 /// 100526830 /// 101929857 /// 101929864
Merged 101929857: 6799 /// 6817 /// 6818 /// 79008 /// 445329 /// 548593 /// 100526830 /// 100526831 /// 101929857 /// 101929864

5725:
5725 /// 100616459
Merged 5725: 5725 /// 100616459

2679:
2678 /// 2679 /// 91227 /// 92086 /// 728441 /// 100132705 /// 102

3439 /// 3447
Merged 3439: 3439 /// 3447

1416:
1415 /// 1416
Merged 1416: 1415 /// 1416

2939:
2938 /// 2939 /// 2940
2938 /// 2939
Merged 2939: 2938 /// 2939 /// 2940

100302736:
51014 /// 100302736
353376 /// 100302736
Merged 100302736: 51014 /// 353376 /// 100302736

6122:
6122 /// 116936 /// 116938
Merged 6122: 6122 /// 116936 /// 116938

55798:
55798 /// 339175
Merged 55798: 55798 /// 339175

54476:
54476 /// 441191
Merged 54476: 54476 /// 441191

84305:
84305 /// 102724112
Merged 84305: 84305 /// 102724112

56114:
5098 /// 8641 /// 9708 /// 26025 /// 56097 /// 56098 /// 56099 /// 56100 /// 56101 /// 56102 /// 56103 /// 56104 /// 56105 /// 56106 /// 56107 /// 56108 /// 56109 /// 56110 /// 56111 /// 56112 /// 56113 /// 56114
Merged 56114: 5098 /// 8641 /// 9708 /// 26025 /// 56097 /// 56098 /// 56099 /// 56100 /// 56101 /// 56102 /// 56103 /// 56104 /// 56105 /// 56106 /// 56107 /// 56108 /// 56109 /// 56110 /// 56111 /// 56112 /// 56113 /// 56114

644634:
57234 /// 388692 /// 644

3493 /// 3494 /// 3500 /// 3502 /// 3507 /// 28396 /// 28442
Merged 28442: 3492 /// 3493 /// 3494 /// 3495 /// 3500 /// 3502 /// 3503 /// 3507 /// 28396 /// 28442

1500:
1500 /// 100528016
Merged 1500: 1500 /// 100528016

441212:
6100 /// 441212
Merged 441212: 6100 /// 441212

100996307:
100996307 /// 101930071
Merged 100996307: 100996307 /// 101930071

728411:
375513 /// 728411 /// 100996497
375513 /// 728411
Merged 728411: 375513 /// 728411 /// 100996497

92086:
2678 /// 91227 /// 92086 /// 728441
2678 /// 2679 /// 91227 /// 92086 /// 728441 /// 100132705 /// 102724197
2678 /// 2679 /// 91227 /// 92086 /// 645367 /// 728441 /// 100132705 /// 102724197
Merged 92086: 2678 /// 2679 /// 91227 /// 92086 /// 645367 /// 728441 /// 100132705 /// 102724197

5395:
5395 /// 441194
5379 /// 5383 /// 5395 /// 441259 /// 100132832
Merged 5395: 5379 /// 5383 /// 5395 /// 441194 /// 441259 /// 100132832

101928349:
202134 /// 285596 /// 100507387 /// 101928349 /// 101930363 /// 102724062 /// 1027252


286042:
55138 /// 286042 /// 389633 /// 729689
Merged 286042: 55138 /// 286042 /// 389633 /// 729689

728712:
30014 /// 728712
30014 /// 64663 /// 728712
30014 /// 64663 /// 728695 /// 728712
Merged 728712: 30014 /// 64663 /// 728695 /// 728712

157769:
57234 /// 157769
Merged 157769: 57234 /// 157769

259291:
259289 /// 259291
Merged 259291: 259289 /// 259291

8034:
8034 /// 100996629
Merged 8034: 8034 /// 100996629

441056:
22947 /// 26582 /// 26583 /// 441056 /// 652301 /// 653543 /// 653544 /// 653545 /// 653548 /// 728410 /// 100133400 /// 100287823 /// 100288289 /// 100288687 /// 100291626 /// 100419743 /// 102723449
22947 /// 26583 /// 441056 /// 652301 /// 653543 /// 653544 /// 653545 /// 653548 /// 728410 /// 100288289 /// 100288687 /// 100291626 /// 100419743 /// 102723449
Merged 441056: 22947 /// 26582 /// 26583 /// 441056 /// 652301 /// 653543 /// 653544 /// 653545 /// 653548 /// 728410 /// 100133400 /// 100287823 /// 100288289 /// 100288687 /// 100291626 /// 100419743 ///


474170:
9884 /// 55073 /// 374819 /// 474170
55073 /// 474170
Merged 474170: 9884 /// 55073 /// 374819 /// 474170

1082:
1082 /// 93659 /// 94027 /// 94115 /// 114335 /// 114336
1082 /// 93659 /// 94027 /// 94115 /// 114335
Merged 1082: 1082 /// 93659 /// 94027 /// 94115 /// 114335 /// 114336

54579:
54575 /// 54576 /// 54577 /// 54578 /// 54579 /// 54600 /// 54657 /// 54658 /// 54659
54576 /// 54579 /// 54600 /// 54658 /// 54659
Merged 54579: 54575 /// 54576 /// 54577 /// 54578 /// 54579 /// 54600 /// 54657 /// 54658 /// 54659

1553:
1548 /// 1549 /// 1553
Merged 1553: 1548 /// 1549 /// 1553

22794:
22794 /// 102466986
Merged 22794: 22794 /// 102466986

8341:
8332 /// 8341
Merged 8341: 8332 /// 8341

2940:
2938 /// 2939 /// 2940
Merged 2940: 2938 /// 2939 /// 2940

26778:
1915 /// 6134 /// 26778
6134 /// 26778
Merged 26778: 1915 /// 6134 /// 26778

101930392:
8440 /// 101930392
Merged 101930392: 8440 /// 101930392

3550:
3550 /// 55374
Merged 3550: 3550 /// 55374

378708:
378708 /// 

In [213]:
common_elements

['51608',
 '284757',
 '5911',
 '80014',
 '54877',
 '8906',
 '2210',
 '3119',
 '5243',
 '27230',
 '5383',
 '4763',
 '8347',
 '100131998',
 '65988',
 '8440',
 '728558',
 '80759',
 '4499',
 '729262',
 '399668',
 '4584',
 '56112',
 '268',
 '6048',
 '1187',
 '3240',
 '65996',
 '26292',
 '2309',
 '8369',
 '53916',
 '503639',
 '7026',
 '2213',
 '6817',
 '160313',
 '599',
 '54811',
 '100293211',
 '101930405',
 '7932',
 '8387',
 '6050',
 '10740',
 '151230',
 '64150',
 '286827',
 '29894',
 '100506840',
 '5379',
 '2966',
 '56940',
 '2575',
 '408050',
 '79933',
 '56136',
 '51522',
 '23617',
 '7626',
 '100861540',
 '3802',
 '26986',
 '27031',
 '4625',
 '7594',
 '100506472',
 '3150',
 '2073',
 '7148',
 '9554',
 '3481',
 '7025',
 '8220',
 '8332',
 '541471',
 '8344',
 '643827',
 '642559',
 '79711',
 '10629',
 '2212',
 '3398',
 '339122',
 '51471',
 '11202',
 '1415',
 '128853',
 '342096',
 '5462',
 '51622',
 '29886',
 '552900',
 '143187',
 '3500',
 '10399',
 '57604',
 '246721',
 '9747',
 '5625',
 '41418

In [210]:
entrez_dups_list = [x for l in entrez_dups_dict.values() for x in l]
print(len(entrez_dups_dict))
print(len(entrez_dups_list))
print(len(set(entrez_dups_list)))

769
1979
1687


In [215]:
entrez_dups_dict

{'284757 /// 729296': ['284757 /// 729296', '284757', '729296'],
 '1485 /// 30848 /// 246100': ['1485 /// 246100',
  '1485 /// 30848 /// 246100',
  '1485',
  '30848',
  '246100'],
 '5911 /// 5912': ['5911 /// 5912', '5911', '5912'],
 '53842 /// 80014': ['53842 /// 80014', '53842', '80014'],
 '54877 /// 400653': ['54877 /// 400653', '54877', '400653'],
 '151230 /// 493911 /// 100526832': ['151230 /// 100526832',
  '151230 /// 493911 /// 100526832',
  '151230',
  '493911',
  '100526832'],
 '10093 /// 26140 /// 100526693': ['10093 /// 26140 /// 100526693',
  '10093 /// 100526693',
  '10093',
  '26140',
  '100526693'],
 '8906 /// 84502': ['8906 /// 84502', '8906', '84502'],
 '4763 /// 101930150': ['4763 /// 101930150', '4763', '101930150'],
 '100131998 /// 101060405': ['100131998 /// 101060405',
  '100131998',
  '101060405'],
 '9659 /// 653513 /// 100996724 /// 100996761 /// 101929792 /// 101930416': ['9659 /// 100996724 /// 100996761 /// 101929792 /// 101930416',
  '9659 /// 653513 /// 10

In [216]:
df_results.reset_index(inplace=True)
for merged_entrez_id, entrez_dups_list in entrez_dups_dict.items():
    # print(merged_entrez_id)
    # print(entrez_dups_list)
    df_results['ENTREZ_GENE_ID'].replace(to_replace=entrez_dups_list,
                                         value=merged_entrez_id,
                                         inplace=True)
#     for entrez_dups in entrez_dups_list:
#         df_results.index.replace(index={entrez_dups:merged_entrez_id},inplace=True)
df_results.set_index('ENTREZ_GENE_ID',inplace=True)
df_results

,GSM60353,GSM60354,GSM60360,GSM60367,GSM60377,GSM60381,GSM60738,GSM60739,GSM60745,GSM60752,...,GSM1054775,GSM1054783,GSM1054787,GSM1054791,GSM1215107,GSM1215109,GSM1215111,GSM1215113,GSM760599,GSM760601
ENTREZ_GENE_ID,,,,,,,,,,,,,,,,,,,,,
780 /// 100616237,1.0,1.0,0.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5982,0.0,0.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3310,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7849,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7318 /// 100847079,1.0,1.0,1.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7067 /// 9572,1.0,0.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11099,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6352,0.0,1.0,1.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [217]:
df_results.fillna(-1).groupby(level=0).max()

,GSM60353,GSM60354,GSM60360,GSM60367,GSM60377,GSM60381,GSM60738,GSM60739,GSM60745,GSM60752,...,GSM1054775,GSM1054783,GSM1054787,GSM1054791,GSM1215107,GSM1215109,GSM1215111,GSM1215113,GSM760599,GSM760601
ENTREZ_GENE_ID,,,,,,,,,,,,,,,,,,,,,
1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
10,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
100,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1000,1.0,1.0,0.0,1.0,0.0,1.0,-1.0,-1.0,-1.0,-1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
10000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
100009676,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
10001,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
10002,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
10003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [191]:
# df_results[df_results.index.str.contains('///')]
df_results.dropna(how='all')

,GSM60353,GSM60354,GSM60360,GSM60367,GSM60377,GSM60381,GSM60738,GSM60739,GSM60745,GSM60752,...,GSM1054775,GSM1054783,GSM1054787,GSM1054791,GSM1215107,GSM1215109,GSM1215111,GSM1215113,GSM760599,GSM760601
ENTREZ_GENE_ID,,,,,,,,,,,,,,,,,,,,,
780 /// 100616237,1.0,1.0,0.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5982,0.0,0.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3310,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7849,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7318 /// 100847079,1.0,1.0,1.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7067,1.0,0.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11099,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6352,0.0,1.0,1.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [132]:
print(cnt)
print(len(entrez_dups_dict))
len(full_entre_id_sets)

540
276


276

In [92]:
import sys

from Bio import Entrez

# *Always* tell NCBI who you are
Entrez.email = "examples1@gmail.com"

def retrieve_annotation(id_list):

    """Annotates Entrez Gene IDs using Bio.Entrez, in particular epost (to
    submit the data to NCBI) and esummary to retrieve the information.
    Returns a list of dictionaries with the annotations."""

    request = Entrez.epost("gene",id=",".join(id_list))
    try:
        result = Entrez.read(request)
    except RuntimeError as e:
        #FIXME: How generate NAs instead of causing an error with invalid IDs?
        print("An error occurred while retrieving the annotations.")
        print("The error returned was %s" % e)
        sys.exit(-1)

    webEnv = result["WebEnv"]
    queryKey = result["QueryKey"]
    data = Entrez.esummary(db="gene", webenv=webEnv, query_key =
            queryKey)
    annotations = Entrez.read(data)

    print("Retrieved %d annotations for %d genes" % (len(annotations),
            len(id_list)))

    return annotations

In [93]:
def print_data(annotation):
    for gene_data in annotation:
        gene_id = gene_data["Id"]
        gene_symbol = gene_data["NomenclatureSymbol"]
        gene_name = gene_data["Description"]
        print("ID: %s - Gene Symbol: %s - Gene Name: %s" % (gene_id, gene_symbol, gene_name))

In [ ]:
annotations =  retrieve_annotation(id_list[0:10])

In [107]:
# for key,annotation in annotations.items():
#     print_data(annotation)
print_data(annotations)

TypeError: string indices must be integers

In [106]:
annotation

DictElement({'DocumentSummary': [DictElement({'Name': 'DDR1', 'Description': 'discoidin domain receptor tyrosine kinase 1', 'Status': '0', 'CurrentID': '0', 'Chromosome': '6', 'GeneticSource': 'genomic', 'MapLocation': '6p21.33', 'OtherAliases': 'CAK, CD167, DDR, EDDR1, HGK2, MCK10, NEP, NTRK4, PTK3, PTK3A, RTK6, TRKE', 'OtherDesignations': 'epithelial discoidin domain-containing receptor 1|CD167 antigen-like family member A|PTK3A protein tyrosine kinase 3A|cell adhesion kinase|mammary carcinoma kinase 10|neuroepithelial tyrosine kinase|neurotrophic tyrosine kinase, receptor, type 4|protein-tyrosine kinase RTK-6|tyrosine kinase DDR|tyrosine-protein kinase CAK', 'NomenclatureSymbol': 'DDR1', 'NomenclatureName': 'discoidin domain receptor tyrosine kinase 1', 'NomenclatureStatus': 'Official', 'Mim': ['600408'], 'GenomicInfo': [{'ChrLoc': '6', 'ChrAccVer': 'NC_000006.12', 'ChrStart': '30880957', 'ChrStop': '30900155', 'ExonCount': '25'}], 'GeneWeight': '17493', 'Summary': 'Receptor tyrosin

In [2]:
projectdir

'/Users/zhzhao/Dropbox/Helikar/pipelines'

In [7]:
agilentfile = 'GSM1054773_US91203659_252800415876_S01_GE1_107_Sep09_1_1.txt'
agilentpath = os.path.join('/Users/zhzhao/Downloads/',agilentfile)

In [3]:
datadir = os.path.join(projectdir,'data','GSE43005_RAW','GPL14550')
datadir

'/Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE43005_RAW/GPL14550'

In [13]:
files = os.listdir(datadir)
gsms = ['GSM1054778','GSM1054786','GSM1054796','GSM1054794']

In [20]:
df_agilent = agilentio.readaiglent(datadir,targets)
df_agilent

Read GSM1054778_US91203659_252800415876_S01_GE1_107_Sep09_2_2.txt.gz 
Read GSM1054786_US91203659_252800415877_S01_GE1_107_Sep09_2_2.txt.gz 
Read GSM1054796_US91203659_252800415878_S01_GE1_107_Sep09_2_4.txt.gz 
Read GSM1054794_US91203659_252800415878_S01_GE1_107_Sep09_2_2.txt.gz 
Array 1 corrected
Array 2 corrected
Array 3 corrected
Array 4 corrected


,Row,Col,ControlType,ProbeName,SystematicName,GSM1054778_US91203659_252800415876_S01_GE1_107_Sep09_2_2.txt,GSM1054786_US91203659_252800415877_S01_GE1_107_Sep09_2_2.txt,GSM1054796_US91203659_252800415878_S01_GE1_107_Sep09_2_4.txt,GSM1054794_US91203659_252800415878_S01_GE1_107_Sep09_2_2.txt
1,1,1,1,GE_BrightCorner,GE_BrightCorner,9.957179,9.120051,11.202998,10.326770
2,1,2,1,DarkCorner,DarkCorner,4.797169,4.897386,4.038584,4.049516
3,1,3,1,DarkCorner,DarkCorner,4.526884,4.797169,1.308718,3.788406
4,1,4,0,A_23_P326296,NM_144987,5.795793,5.652573,4.968230,5.337283
5,1,5,0,A_24_P287941,NM_013290,4.797169,4.822884,0.707404,4.829241
6,1,6,0,A_24_P325046,ENST00000322831,5.002388,4.945000,4.632444,4.421431
7,1,7,0,A_23_P200404,NM_001625,5.314806,5.291972,5.062986,5.428042
8,1,8,0,A_19_P00800513,lincRNA:chr7:226042-232442_R,5.831067,5.831067,5.964122,6.433427
9,1,9,0,A_23_P15619,NM_032391,4.429823,5.019169,0.805458,1.700435
10,1,10,0,A_33_P3402354,ENST00000238571,4.841873,4.603064,3.227525,3.788406


In [15]:
txts = []
gzs = []
for gsm in gsms:
    for file in files:
        if gsm in file:
            gzs.append(file)
            txts.append(file[0:-3])
print(gzs)
print(txts)

['GSM1054778_US91203659_252800415876_S01_GE1_107_Sep09_2_2.txt.gz', 'GSM1054786_US91203659_252800415877_S01_GE1_107_Sep09_2_2.txt.gz', 'GSM1054796_US91203659_252800415878_S01_GE1_107_Sep09_2_4.txt.gz', 'GSM1054794_US91203659_252800415878_S01_GE1_107_Sep09_2_2.txt.gz']
['GSM1054778_US91203659_252800415876_S01_GE1_107_Sep09_2_2.txt', 'GSM1054786_US91203659_252800415877_S01_GE1_107_Sep09_2_2.txt', 'GSM1054796_US91203659_252800415878_S01_GE1_107_Sep09_2_4.txt', 'GSM1054794_US91203659_252800415878_S01_GE1_107_Sep09_2_2.txt']


In [17]:
targets = pd.DataFrame(gzs,columns=['FileName'],index=txts)
targets

,FileName
GSM1054778_US91203659_252800415876_S01_GE1_107_Sep09_2_2.txt,GSM1054778_US91203659_252800415876_S01_GE1_107...
GSM1054786_US91203659_252800415877_S01_GE1_107_Sep09_2_2.txt,GSM1054786_US91203659_252800415877_S01_GE1_107...
GSM1054796_US91203659_252800415878_S01_GE1_107_Sep09_2_4.txt,GSM1054796_US91203659_252800415878_S01_GE1_107...
GSM1054794_US91203659_252800415878_S01_GE1_107_Sep09_2_2.txt,GSM1054794_US91203659_252800415878_S01_GE1_107...


In [18]:
agilentstring="""
readaiglent <- function(addr,targets){
    crd <- getwd()
    setwd(addr)
    # targets <- dir(".", "txt.gz")
    x <- read.maimages(targets, "agilent", green.only = TRUE)
    setwd(crd)
    y <- backgroundCorrect(x,method="normexp") 
    y <- normalizeBetweenArrays(y,method="quantile") 
    yo <- c(y$genes,as.data.frame(y$E))
    ydf <- data.frame(yo)
    return(ydf)
}
"""

agilentio = SignatureTranslatedAnonymousPackage(agilentstring, "agilentio")


In [27]:
df = df_agilent.drop(columns=['Row','Col'])
df_neg = df[df['ControlType']==-1]
df_cutoff = df_neg.drop(columns=['ControlType']).quantile(0.95,axis=0)
# df_agilent.quantile(0.95,axis=0)

In [25]:
list(df)

['ControlType',
 'ProbeName',
 'SystematicName',
 'GSM1054778_US91203659_252800415876_S01_GE1_107_Sep09_2_2.txt',
 'GSM1054786_US91203659_252800415877_S01_GE1_107_Sep09_2_2.txt',
 'GSM1054796_US91203659_252800415878_S01_GE1_107_Sep09_2_4.txt',
 'GSM1054794_US91203659_252800415878_S01_GE1_107_Sep09_2_2.txt']

In [31]:
# for idx in df_cutoff.index.tolist():
    
df_bool = df.loc[df['ControlType']==0,df_cutoff.index.tolist()].gt(1.1*df_cutoff,axis=1)

In [40]:
df.loc[df_bool[df_bool.all(axis=1)].index,]

,ControlType,ProbeName,SystematicName,GSM1054778_US91203659_252800415876_S01_GE1_107_Sep09_2_2.txt,GSM1054786_US91203659_252800415877_S01_GE1_107_Sep09_2_2.txt,GSM1054796_US91203659_252800415878_S01_GE1_107_Sep09_2_4.txt,GSM1054794_US91203659_252800415878_S01_GE1_107_Sep09_2_2.txt
12,0,A_32_P98683,NM_005937,8.402705,6.799968,8.215184,7.540539
20,0,A_24_P186124,NM_182501,6.494704,6.056432,6.056432,6.107019
21,0,A_23_P369983,NM_174905,7.553556,6.890909,6.723531,6.522433
25,0,A_24_P37441,NM_002610,6.200713,6.304749,5.926071,5.821532
26,0,A_23_P20980,NM_001916,6.553484,6.644621,6.410290,6.433427
29,0,A_23_P339687,NM_138330,6.618246,6.890909,6.763916,6.538042
38,0,A_19_P00800449,lincRNA:chr8:145986196-145991421_F,7.758576,7.986144,7.630653,6.796728
40,0,A_23_P97700,NM_006472,11.876106,12.338612,11.266744,10.398466
44,0,A_23_P115955,NM_181515,7.305405,8.358335,7.161036,7.924014
63,0,A_24_P410587,NR_026903,6.976455,6.920984,6.646422,6.559232


In [35]:
df_bool

,GSM1054778_US91203659_252800415876_S01_GE1_107_Sep09_2_2.txt,GSM1054786_US91203659_252800415877_S01_GE1_107_Sep09_2_2.txt,GSM1054796_US91203659_252800415878_S01_GE1_107_Sep09_2_4.txt,GSM1054794_US91203659_252800415878_S01_GE1_107_Sep09_2_2.txt
4,True,False,False,False
5,False,False,False,False
6,False,False,False,False
7,False,False,False,False
8,True,False,True,True
9,False,False,False,False
10,False,False,False,False
11,False,False,False,False
12,True,True,True,True
13,True,True,False,False


In [5]:
from instruments import *
gsms = ['GSM1054778','GSM1054786','GSM1054796','GSM1054794']
df_results = readagilent(datadir,gsms)
df_results

Read GSM1054778_US91203659_252800415876_S01_GE1_107_Sep09_2_2.txt.gz 
Read GSM1054786_US91203659_252800415877_S01_GE1_107_Sep09_2_2.txt.gz 
Read GSM1054796_US91203659_252800415878_S01_GE1_107_Sep09_2_4.txt.gz 
Read GSM1054794_US91203659_252800415878_S01_GE1_107_Sep09_2_2.txt.gz 
Array 1 corrected
Array 2 corrected
Array 3 corrected
Array 4 corrected


,ProbeName,SystematicName,Express
4,A_23_P326296,NM_144987,0.0
5,A_24_P287941,NM_013290,0.0
6,A_24_P325046,ENST00000322831,0.0
7,A_23_P200404,NM_001625,0.0
8,A_19_P00800513,lincRNA:chr7:226042-232442_R,0.0
9,A_23_P15619,NM_032391,0.0
10,A_33_P3402354,ENST00000238571,0.0
11,A_33_P3338798,XM_003120083,0.0
12,A_32_P98683,NM_005937,1.0
13,A_23_P137543,NM_152493,0.0


In [7]:
import mygene
mg = mygene.MyGeneInfo()
mg.metadata['available_fields']

TypeError: 'method' object is not subscriptable

In [9]:
mg.get_fields()

{'AnimalQTLdb': {'type': 'text', 'index': True},
 'FLYBASE': {'type': 'keyword', 'index': True},
 'HGNC': {'type': 'keyword', 'index': True},
 'HPRD': {'type': 'keyword', 'index': True},
 'MGI': {'type': 'keyword', 'index': True},
 'MIM': {'type': 'keyword', 'index': True},
 'RATMAP': {'type': 'keyword', 'index': True},
 'RGD': {'type': 'keyword', 'index': True},
 'TAIR': {'type': 'keyword', 'index': True},
 'Vega': {'type': 'text', 'index': True},
 'WormBase': {'type': 'keyword', 'index': True},
 'Xenbase': {'type': 'keyword', 'index': True},
 'ZFIN': {'type': 'keyword', 'index': True},
 'accession': {'type': 'object', 'index': True},
 'accession.genomic': {'type': 'text', 'index': True},
 'accession.protein': {'type': 'text',
  'index': True,
  'analyzer': 'refseq_analyzer',
  'searched_by_default': True},
 'accession.rna': {'type': 'text',
  'index': True,
  'analyzer': 'refseq_analyzer',
  'searched_by_default': True},
 'accession.translation': {'type': 'object', 'index': True},
 '

In [11]:
dir(mg)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_annotation_endpoint',
 '_annotations_generator',
 '_cached',
 '_clear_cache',
 '_dataframe',
 '_default_cache_file',
 '_default_url',
 '_delay',
 '_docstring_obj',
 '_entity',
 '_fetch_all',
 '_format_list',
 '_from_cache_notification',
 '_get',
 '_get_fields',
 '_getannotation',
 '_getannotations',
 '_getannotations_inner',
 '_max_query',
 '_metadata',
 '_metadata_endpoint',
 '_metadata_fields_endpoint',
 '_optionally_plural_object_type',
 '_pkg_user_agent_header',
 '_post',
 '_query',
 '_query_endpoint',
 '_querymany',
 '_querymany_inner',
 '_repeated_query',
 '_repeated_query_old',
 '_scroll_size',
 '_set_caching',
 

In [13]:
mg.getgenes(df_results.ProbeName.tolist(), fields="entrezgene")

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
querying 19001-20000...done.
querying 20001-21000...done.
querying 21001-22000...done.
querying 22001-23000...done.
querying 23001-24000...done.
querying 24001-25000...done.
querying 25001-26000...done.
querying 26001-27000...done.
querying 27001-28000...done.
querying 28001-29000...done.
querying 29001-30000...done.
querying 30001-31000...done.
querying 31001-32000...done.
querying 32001-33000...done.
querying 33001-34000...done.
querying 34001-35000...done.
queryin

[{'query': 'A_23_P326296', 'notfound': True},
 {'query': 'A_24_P287941', 'notfound': True},
 {'query': 'A_24_P325046', 'notfound': True},
 {'query': 'A_23_P200404', 'notfound': True},
 {'query': 'A_19_P00800513', 'notfound': True},
 {'query': 'A_23_P15619', 'notfound': True},
 {'query': 'A_33_P3402354', 'notfound': True},
 {'query': 'A_33_P3338798', 'notfound': True},
 {'query': 'A_32_P98683', 'notfound': True},
 {'query': 'A_23_P137543', 'notfound': True},
 {'query': 'A_19_P00803040', 'notfound': True},
 {'query': 'A_23_P117852', 'notfound': True},
 {'query': 'A_33_P3285585', 'notfound': True},
 {'query': 'A_24_P328231', 'notfound': True},
 {'query': 'A_33_P3415668', 'notfound': True},
 {'query': 'A_23_P73609', 'notfound': True},
 {'query': 'A_24_P186124', 'notfound': True},
 {'query': 'A_23_P369983', 'notfound': True},
 {'query': 'A_23_P325676', 'notfound': True},
 {'query': 'A_24_P37441', 'notfound': True},
 {'query': 'A_23_P20980', 'notfound': True},
 {'query': 'A_23_P109184', 'not

In [32]:
from bioservices import BioDBNet
s = BioDBNet()
input_db = 'Agilent ID'
output_db = ['Gene ID','Ensembl Gene ID']
input_values = df_results.ProbeName.tolist()

df_maps = pd.DataFrame([],columns=output_db)
df_maps.index.name=input_db
for i in range(0,len(input_values),500):
    print('retrieve {}:{}'.format(i,min(i+500,len(input_values))))
    df_test = s.db2db(input_db, output_db, input_values[i:min(i+500,len(input_values))], 9606)
    df_maps = pd.concat([df_maps, df_test], sort=False)
df_maps

INFO    [bioservices:BioDBNet]:  Initialising BioDBNet service (REST)


retrieve 0:500
retrieve 500:1000
retrieve 1000:1500
retrieve 1500:2000
retrieve 2000:2500
retrieve 2500:3000
retrieve 3000:3500
retrieve 3500:4000
retrieve 4000:4500
retrieve 4500:5000
retrieve 5000:5500
retrieve 5500:6000
retrieve 6000:6500
retrieve 6500:7000
retrieve 7000:7500
retrieve 7500:8000
retrieve 8000:8500
retrieve 8500:9000
retrieve 9000:9500
retrieve 9500:10000
retrieve 10000:10500
retrieve 10500:11000
retrieve 11000:11500
retrieve 11500:12000
retrieve 12000:12500
retrieve 12500:13000
retrieve 13000:13500
retrieve 13500:14000
retrieve 14000:14500
retrieve 14500:15000
retrieve 15000:15500
retrieve 15500:16000
retrieve 16000:16500
retrieve 16500:17000
retrieve 17000:17500
retrieve 17500:18000
retrieve 18000:18500
retrieve 18500:19000
retrieve 19000:19500
retrieve 19500:20000
retrieve 20000:20500
retrieve 20500:21000
retrieve 21000:21500
retrieve 21500:22000
retrieve 22000:22500
retrieve 22500:23000
retrieve 23000:23500
retrieve 23500:24000
retrieve 24000:24500
retrieve 24500:

,Gene ID,Ensembl Gene ID
Agilent ID,,
A_23_P326296,199746,ENSG00000161265
A_24_P287941,29893,ENSG00000131470
A_24_P325046,84186,ENSG00000147905
A_23_P200404,204,ENSG00000004455
A_19_P00800513,-,ENSG00000249852
A_23_P15619,84366,ENSG00000159182
A_33_P3402354,56252,ENSG00000119596
A_33_P3338798,-,ENSG00000253542
A_32_P98683,4302,ENSG00000275023//ENSG00000275851


In [29]:
# df_test.to_csv('')
range(0,len(input_values),500)

range(0, 58717, 500)

In [33]:
df_maps.to_csv('./agilent_ID_to_ENTREZ_GENE_ID.csv')

In [3]:
df_maps = pd.read_csv('./agilent_ID_to_ENTREZ_GENE_ID.csv')

In [8]:
agilentids = df_maps['Agilent ID'].unique()
agilentids.shape

(42405,)